In [1]:
# im just grabing different part of your scripts and call len() functions

import sys
#!{sys.executable} -m pip install pymongo
#!{sys.executable} -m pip install pyyaml
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install moment
#!{sys.executable} -m pip install tldextract
import os
print(os.getcwd())

/home/zilin/habitlab-data-analysis


In [2]:
from dataUtil import *
import os
print(os.getcwd())
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret
import numpy as np
from browser_libs import *
from dataUtil import *
client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("MOBILE_NAME")]
ext_client = MongoClient(getsecret("EXT_URI"))
ext_db = ext_client[getsecret("DB_NAME")]
# Get all synced accounts and their respective users.
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))

/home/zilin/habitlab-data-analysis


* Participants in this study consisted of new HabitLab users who installed either the HabitLab Chrome extension or An-droid app over a period of N weeks in July through September 2018.
* N users installed the HabitLab Chrome version overthe course of our experiment and consented to our research protocol. 
* N users did so for the Android version. 
* N installed both and signed in with their Google accounts, allowing us to analyze their usage across devices. 
* We discarded partici-pants who were not new users of HabitLab, since some userswere reinstalls or new devices for existing users. We also discarded participants who did not complete the onboard-ing process, or who uninstalled the system before they saw their first intervention. 
* This left us with N participants forChrome, N participants for Android, and N participants forwhom we could analyze usage across both.

# (Multiple installs & not onboarded) not filtered

In [3]:
#browser_list_with_multi_install = get_users_with_goal_frequency_set()
num_browser  = len(get_users_with_goal_frequency_set())
num_android = len([x for x in accounts if x["android"] != []])
num_both = len([x for x in accounts if x["android"] != [] and x["browser"] != []])
print("browser: ", num_browser)
print("android: ", num_android)
print("both: ", num_both)


browser:  2058
android:  995
both:  199


# Filtered

In [4]:
get_last_intervention_seen("8916c882cdc10370b3f3d205")

'facebook:remove_news_feed'

In [5]:
get_did_user_finish_onboarding("8916c882cdc10370b3f3d205")

True

In [6]:
browser_list = get_valid_user_list()
def filter_onboard_seen_intervention(id):
    print(get_did_user_finish_onboarding(id)) 
    if get_did_user_finish_onboarding(id) and get_last_intervention_seen(id) != "null":
        return True
    else:
        return False
    
browser_list_filtered = list(filter(filter_onboard_seen_intervention, browser_list))
num_browser_filtered = len(browser_list_filtered)
accounts_filtered = []
for account in accounts:
    if account["browser"] == []:
        accounts_filtered.append(account)
        continue
    if account["browser"][0] in browser_list_filtered:
        accounts_filtered.append(account)
        
num_both_filtered = len([x for x in accounts_filtered if len(x["browser"]) == 1 and len(x["android"]) == 1])
num_android_filtered = len([x for x in accounts_filtered if len(x["android"]) == 1])

False
False
True
True
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
True
True
False
True
False
True
False
False
True
True
False
True
True
True
False
True
True
False
True
False
True
False
True
True
True
True
True
True
True
True
False
False
True
True
True
False
False
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
False
False
True
True
True
False
True
True
True
False
True
True
True
False
False
False
False
True
False
True
True
False
True
False
True
False
True
True
True
False
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
False
True
True
True
True
True
False
False
True
True
True
True
False
True
True
False
True
True
True
True
False
False
False
True
False
False
False
True
False
True
False
True
True
True
True
True
False
True
False
True
False
False
True
True
False
Tru

In [8]:
print("browser: ", num_browser_filtered)
print("android: ", num_android_filtered)
print("both: ", num_both_filtered)

browser:  985
android:  822
both:  51


# Session Data & Time

In [9]:
frequency_goals = []
for user in browser_list_filtered:
    raw = parse_url_as_json("http://localhost:5001/printcollection?collection=" + user + "_synced:goal_frequencies")
    frequency_goals.append(raw[0])
#print(frequency_goals)
timestamps = [x['timestamp'] for x in frequency_goals]
latest = max(timestamps)
earliest = min(timestamps)
print(latest)
print(earliest)
print("the study spent (weeks): ", (latest - earliest) * 1.6534e-9)

1537239669790
1533480294751
the study spent (weeks):  6.215750689482601


### Android Session

In [10]:
# Get all sessions:
sessions = {}
for account in accounts_filtered:
    for user in account["android"]:
        for session in db[user + "_sessions"].find({"enabled": {"$exists": True}}):
            if user not in sessions:
                sessions[user] = {}
            domain = session["domain"]
            if domain not in sessions[user]:
                sessions[user][domain] = []
            sessions[user][domain].append(session)
            
#print(sessions['bdc472cb840f151f72f88ec3'])
def countSessions(sessions):
    total = 0
    for domain in sessions:
        total += len(sessions[domain])
        
    return total

android_userids = [x['android'][0] for x in accounts_filtered if len(x["browser"]) <= 1 and len(x["android"]) == 1]
both_userids_android = [x['android'][0] for x in accounts_filtered if len(x["browser"]) == 1 and len(x["android"]) == 1]

total_a = 0
for userid in android_userids:
    if userid in sessions:
        total_a += countSessions(sessions[userid])


print("total number of sessions for android =", total_a)



total number of sessions for android = 2782465


In [11]:

total_a_sync = 0
for userid in both_userids_android:
    if userid in sessions:
        total_a_sync += countSessions(sessions[userid])

print("total number of sessions for android synced =", total_a_sync)

total number of sessions for android synced = 304655


### Browser Session

In [12]:
#print(get_total_number_of_sessions_for_user("8916c882cdc10370b3f3d205"))
print("total number of sessions for browser =", sum(map(get_total_number_of_sessions_for_user, browser_list_filtered)))
both_userids = [x['browser'][0] for x in accounts if len(x["browser"]) == 1 and len(x["android"]) == 1]
print("total number of sessions for browser synced =", sum(map(get_total_number_of_sessions_for_user, both_userids)))

total number of sessions for browser = 1127486
total number of sessions for browser synced = 217321


In [13]:
print(get_total_number_of_sessions_for_user("8916c882cdc10370b3f3d205"))

28


In [14]:
browser_list_filtered

['2b5972416d3df31ad7b1c25f',
 '46f96a6ab8753a767bdd1b1b',
 '30134f4f9d24141ae0bca891',
 '11f367d7c4c9743aabbda7ff',
 'e6be8edae93ade10b9c7f78d',
 '14fd34ba7978f5c31557000d',
 '4af81d6f6b903b27f34c5f41',
 '921b2a944cc8faa54efa7e89',
 '9514fe891b9ee2a083cb7cb6',
 '3aa9e9b4f04c7f4d5a61b58e',
 'dd6e625a532e9dda1fcfc08e',
 '17e7de4f422a37ce6b911178',
 '68ce22b94204c4959b3bca47',
 'b2f7cba6e365326b0867adaa',
 '5655fa9ee5d384a75feacd10',
 'fd208d6ea1f97ac6e8d3c647',
 '790019388193eb88b81aca56',
 '30f66edc6f4522a2466d5ce0',
 '6b719706061bd70657d8908a',
 'befa7276e957100eebe6bf2c',
 'c8b0e3c4f63d4dc18538d44f',
 '8aa90cc3b2a3d1318b82999a',
 'd0653b1b154105cae6b20f5a',
 'ed86421a4167fd4d84b5dc4a',
 '8e445b95a635466551c7fe9a',
 '7de231e89272243eef3728c9',
 '413790b746780d4712adad4b',
 '31a2b705e857b148ca832ca6',
 'c6424c6fc44ce87b7313e4ea',
 'f6ac752bff889550a3b3f023',
 'eba693ff2cc67e6a1530697d',
 '2fbd7c5f1fdb8daea83144a8',
 '2d70dfa69ebe2178669b477c',
 '93f5b666051403f6dedc9490',
 '3d09d074a8cd

# Participant Text